# ДЗ №2 Оценка качества Visual Language Model в задачах e-com
### Выполнила: Гранкина Елизавета
Вы работаете в R&D-подразделении крупного e-commerce-сервиса.  
В компании уже используются разные модели для:
- классификации товаров по категориям,
- генерации описаний и заголовков,
- извлечения признаков для поиска и рекомендаций.

Хочется заменить этот зоопарк моделей на **единую мультимодальную модель (VLM)**,  
которая могла бы решать несколько задач сразу.

Ваша задача — построить **мини-бенчмарки** для оценки мультимодальных моделей  на реальном датасете товаров, провести сравнение моделей и сделать вывод,  
какая модель лучше подходит для внедрения в e-commerce-сценарий.


## Исходные данные

В рамках задания предлагается использовать датасет [**AMAZON-Products-2023**](https://huggingface.co/datasets/milistu/AMAZON-Products-2023),  
содержащий данные о товарах (изображения, названия, описания, категории, цены и пр.).  

Размер датасета — более 100 000 объектов,  
поэтому вам необходимо выбрать **подмножество 1000–2000 объектов** для экспериментов. Какие именно объекты выбирать - необходимо определить Вам, чтобы бенчмарки были как можно более репрезентативными.

### Выберем подмножество 1000–2000 объектов для экспериментов

Импорты

In [4]:
import os, warnings
from transformers.utils import logging as hf_logging
from datasets import load_dataset, concatenate_datasets, DatasetDict, ClassLabel
from collections import Counter
import re, math
from pathlib import Path
import pandas as pd, json

os.environ["TRANSFORMERS_VERBOSITY"] = "error"
hf_logging.set_verbosity_error()

warnings.filterwarnings("ignore", message=r"The image processor of type `Qwen2VLImageProcessor`.*")
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

'expandable_segments:True'

In [ ]:
def _system_info():
    info = {
        "transformers": None,
        "torch": None,
        "cuda_available": False,
        "cuda_device": None,
        "env": {k:v for k,v in os.environ.items() if k in ["CUDA_VISIBLE_DEVICES"]},
    }
    try:
        import transformers, torch
        info["transformers"] = transformers.__version__
        info["torch"] = torch.__version__
        info["cuda_available"] = torch.cuda.is_available()
        if torch.cuda.is_available():
            dev = torch.cuda.current_device()
            info["cuda_device"] = torch.cuda.get_device_name(dev)
    except Exception:
        pass
    return info
_system_info()

{'transformers': '4.57.1',
 'torch': '2.8.0+cu126',
 'cuda_available': True,
 'cuda_device': 'NVIDIA A100-SXM4-80GB',
 'env': {}}

Загрузка датасета

In [9]:
HF_DATASET = "milistu/AMAZON-Products-2023"
REVISION = None
TARGET_TOTAL = 1500 # целевой размер поднабора (в диапазоне 1000–2000)
TOP_K_LABELS = 10 # сколько крупных классов взять
MIN_COUNT_PER_LABEL = 200 # минимальный размер класса, чтобы попасть в топ
RANDOM_SEED = 42

out_dir = Path("./content/sample_data/amazon_subset_hf"); out_dir.mkdir(exist_ok=True, parents=True)

ds = load_dataset(HF_DATASET, revision=REVISION)
split_name = "train" if "train" in ds else list(ds.keys())[0]
ds = ds[split_name]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data-00000-of-00004.arrow:   0%|          | 0.00/437M [00:00<?, ?B/s]

data-00001-of-00004.arrow:   0%|          | 0.00/432M [00:00<?, ?B/s]

data-00002-of-00004.arrow:   0%|          | 0.00/422M [00:00<?, ?B/s]

data-00003-of-00004.arrow:   0%|          | 0.00/423M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/117243 [00:00<?, ? examples/s]

Обработка и фильтры датасета для создания подвыборки

In [11]:
def filename_to_label(fname: str) -> str:
    if not isinstance(fname, str): return "Unknown"
    s = re.sub(r"^meta_", "", fname).replace("_", " ").strip()
    return s or "Unknown"

def derive_label(example):
    main = example.get("main_category", None)
    if main and str(main).strip():
        lab = str(main).strip()
    else:
        lab = filename_to_label(example.get("filename", "meta_Unknown"))
    example["label"] = lab
    return example

def clean_text(example):
    t = (example.get("title") or "").strip()
    d = (example.get("description") or "").replace("\n", " ").strip() if example.get("description") else ""
    txt = f"{t}. {d}" if d else t
    example["text"] = txt[:300]
    return example

def extract_brand(example):
    store = example.get("store", None)
    if not store or not isinstance(store, str):
        example["brand"] = None; return example
    s = store.strip()
    if "Format:" in s:
        s = s.split("Format:")[0].strip()
    if len(s.split()) > 6:
        example["brand"] = None
    else:
        example["brand"] = s if s else None
    return example

ds = ds.map(derive_label, desc="Derive label")
ds = ds.map(clean_text, desc="Build text")
ds = ds.map(extract_brand, desc="Extract brand")

Derive label:   0%|          | 0/117243 [00:00<?, ? examples/s]

Build text:   0%|          | 0/117243 [00:00<?, ? examples/s]

Extract brand:   0%|          | 0/117243 [00:00<?, ? examples/s]

Фильтр по длине

In [ ]:
def has_quality(example):
    title_ok = isinstance(example.get("title"), str) and len(example["title"].strip()) >= 5
    image_ok = isinstance(example.get("image"), str) and len(example["image"].strip()) > 5
    return title_ok and image_ok

ds = ds.filter(has_quality, desc="Filter quality")

Filter quality:   0%|          | 0/117243 [00:00<?, ? examples/s]

Ищем топ-метки по размеру и балансируем отбор

In [12]:
counts = Counter(ds["label"])
top_labels = [lab for lab, cnt in counts.most_common() if cnt >= MIN_COUNT_PER_LABEL][:TOP_K_LABELS]

per_label = max(1, TARGET_TOTAL // len(top_labels))
parts = []
for lab in top_labels:
    sub = ds.filter(lambda e: e["label"] == lab)
    # перемешаем и возьмём per_label (или меньше, если не хватает)
    sub = sub.shuffle(seed=RANDOM_SEED)
    take = min(len(sub), per_label)
    parts.append(sub.select(range(take)))

subset = concatenate_datasets(parts).shuffle(seed=RANDOM_SEED)

Filter:   0%|          | 0/117243 [00:00<?, ? examples/s]

Filter:   0%|          | 0/117243 [00:00<?, ? examples/s]

Filter:   0%|          | 0/117243 [00:00<?, ? examples/s]

Filter:   0%|          | 0/117243 [00:00<?, ? examples/s]

Filter:   0%|          | 0/117243 [00:00<?, ? examples/s]

Filter:   0%|          | 0/117243 [00:00<?, ? examples/s]

Filter:   0%|          | 0/117243 [00:00<?, ? examples/s]

Filter:   0%|          | 0/117243 [00:00<?, ? examples/s]

Filter:   0%|          | 0/117243 [00:00<?, ? examples/s]

Filter:   0%|          | 0/117243 [00:00<?, ? examples/s]

Проверяем длину выбранного поднабора

In [13]:
# Если меньше 1000 — добавим ещё немного
if len(subset) < 1000:
    need = 1000 - len(subset)
    extra_per_label = math.ceil(need / len(top_labels))
    extras = []
    for lab in top_labels:
        sub = ds.filter(lambda e: e["label"] == lab).shuffle(seed=RANDOM_SEED)
        take = min(len(sub), extra_per_label)
        extras.append(sub.select(range(take)))
    subset = concatenate_datasets([subset] + extras).shuffle(seed=RANDOM_SEED)

# Ограничим верхнюю границу 2000
if len(subset) > 2000:
    subset = subset.select(range(2000))

print(f"Поднабор: {len(subset)} | классы: {sorted(set(subset['label']))}")

Поднабор: 1500 | классы: ['AMAZON FASHION', 'All Electronics', 'Amazon Home', 'Automotive', 'Cell Phones & Accessories', 'Clothing Shoes and Jewelry', 'Health & Personal Care', 'Home and Kitchen', 'Office Products', 'Tools & Home Improvement']


Сохранение итогового поднабора

In [14]:
# 1) Фиксируем порядок меток (чтобы потом label_id совпадал)
labels_sorted = sorted(set(subset["label"]))
label2id = {lab: i for i, lab in enumerate(labels_sorted)}

# 2) Добавляем числовой индекс метки
subset = subset.map(lambda e: {"label_idx": label2id[e["label"]]})
subset = subset.cast_column("label_idx", ClassLabel(names=labels_sorted))

# 3) Сплиты по label_idx
tmp = subset.train_test_split(test_size=0.15, seed=RANDOM_SEED, stratify_by_column="label_idx")
train_val = tmp["train"]
test = tmp["test"]
tv = train_val.train_test_split(test_size=0.1765, seed=RANDOM_SEED, stratify_by_column="label_idx")  # 0.1765 ≈ 15/85
train, val = tv["train"], tv["test"]
final = DatasetDict(train=train, validation=val, test=test)

# 4) Проставим label_id
def add_label_id(example):
    example["label_id"] = label2id[example["label"]]
    return example

for split in final:
    final[split] = final[split].map(add_label_id)

# Сохраним набор и метки
labels_sorted = sorted(set(subset["label"]))
label2id = {lab: i for i, lab in enumerate(labels_sorted)}

def add_label_id(example):
    example["label_id"] = label2id[example["label"]]
    return example

for split in final:
    final[split] = final[split].map(add_label_id)
keep_cols = [
    "parent_asin", "text", "image", "label", "label_id",
    "price", "average_rating", "rating_number",
    "date_first_available", "filename", "main_category", "brand"
]

def ensure_cols(ds_split, cols):
    # 1) добавить отсутствующие колонки как None
    missing = [c for c in cols if c not in ds_split.column_names]
    for m in missing:
        ds_split = ds_split.add_column(m, [None] * len(ds_split))
    # 2) удалить лишние колонки
    extras = [c for c in ds_split.column_names if c not in cols]
    if extras:
        ds_split = ds_split.remove_columns(extras)
    # 3) порядок колонок зададим уже после to_pandas()
    return ds_split

for split in final:
    final[split] = ensure_cols(final[split], keep_cols)
    final[split] = final[split].add_column("split", [split]*len(final[split]))

manifest = concatenate_datasets([final["train"], final["validation"], final["test"]])

# финальный порядок колонок в pandas
df = manifest.to_pandas()
df = df[keep_cols + ["split"]]

manifest_path = out_dir / "subset_balanced_1k_2k.csv"
labelmap_path = out_dir / "label_map.json"

df.to_csv(manifest_path, index=False)
with open(labelmap_path, "w", encoding="utf-8") as f:
    json.dump(label2id, f, ensure_ascii=False, indent=2)

print(f"Saved:\n- {manifest_path}\n- {labelmap_path}")

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1049 [00:00<?, ? examples/s]

Map:   0%|          | 0/226 [00:00<?, ? examples/s]

Map:   0%|          | 0/225 [00:00<?, ? examples/s]

Map:   0%|          | 0/1049 [00:00<?, ? examples/s]

Map:   0%|          | 0/226 [00:00<?, ? examples/s]

Map:   0%|          | 0/225 [00:00<?, ? examples/s]

Saved:
- content/sample_data/amazon_subset_hf/subset_balanced_1k_2k.csv
- content/sample_data/amazon_subset_hf/label_map.json


Вычитка поднабора локально

In [6]:
csv_candidates = [Path("/content/sample_data/subset_balanced_1k_2k.csv")]
map_candidates = [Path("/content/sample_data/label_map.json")]

manifest_path = next((p for p in csv_candidates if p.exists()), None)
labelmap_path = next((p for p in map_candidates if p.exists()), None)
assert manifest_path, "Не найден subset_balanced_1k_2k.csv"
assert labelmap_path, "Не найден label_map.json"

df = pd.read_csv(manifest_path)
with open(labelmap_path, "r", encoding="utf-8") as f:
    label2id = json.load(f)

Инфо о поднаборе

In [25]:
# сводка по сплитам (работает и с 'val', и с 'validation')
stats = (
    df.groupby(["label", "split"])
      .size()
      .reset_index(name="count")
      .pivot_table(index="label", columns="split", values="count", aggfunc="sum", fill_value=0)
      .reset_index()
)
# доли по строке
split_cols = [c for c in stats.columns if c not in ["label"]]
row_sum = stats[split_cols].sum(axis=1).replace({0: pd.NA})
for c in split_cols:
    stats[c + "_pct"] = (stats[c] / row_sum).round(4)

print(f"Loaded: {manifest_path}  rows={len(df)}  classes={df['label'].nunique()}")
display(stats)

Loaded: /content/sample_data/subset_balanced_1k_2k.csv  rows=1500  classes=10


split,label,test,train,validation,test_pct,train_pct,validation_pct
0,AMAZON FASHION,22,105,23,0.1467,0.7000,0.1533
1,All Electronics,23,105,22,0.1533,0.7000,0.1467
2,Amazon Home,23,104,23,0.1533,0.6933,0.1533
3,Automotive,22,105,23,0.1467,0.7000,0.1533
4,Cell Phones & Accessories,22,105,23,0.1467,0.7000,0.1533
5,Clothing Shoes and Jewelry,22,105,23,0.1467,0.7000,0.1533
6,Health & Personal Care,23,105,22,0.1533,0.7000,0.1467
7,Home and Kitchen,22,105,23,0.1467,0.7000,0.1533
8,Office Products,23,105,22,0.1533,0.7000,0.1467
9,Tools & Home Improvement,23,105,22,0.1533,0.7000,0.1467


## Часть 1. Классификация / регрессия

1. Сформулируйте задачу, которую можно оценить классическими метриками  
   (например, классификация категории товара, предсказание диапазона цены, определение бренда и т.д.).

2. Подготовьте данные (изображения, названия, метки) в удобном для инференса формате.

3. Примените 2–3 мультимодальные модели (например, Qwen2.5-VL, LLaVA-OneVision, InternVL). Обратите внимание на время инференса каждой модели, оцените необходимые ресурсы заранее, чтобы не упереться по времени в дедлайн.

4. Рассчитайте **эвристики и метрики** для оценки качества модели:
   - accuracy, F1, MSE, cosine similarity, и др.
   - опишите, почему вы выбрали именно эти метрики.

5. Сделайте **анализ результатов**:
   - где модели ошибаются;
   - какие категории / признаки наиболее сложны;
   - какие факторы влияют на качество.

---

### 1. Постановка задачи

**Задача:** мультимодальная классификация категории товара (10 классов).

**Вход:** изображение товара (image_url) + короткий текст (title + краткое описание).

**Выход:** одна метка из фиксированного списка 10 категорий (см. label_map.json).

### 2. Подготовка данных

In [7]:
from __future__ import annotations

# Входные файлы
csv_path = Path("sample_data/subset_balanced_1k_2k.csv")
map_path = Path("sample_data/label_map.json")
assert csv_path.exists(), f"Не найден файл: {csv_path}"
assert map_path.exists(), f"Не найден файл: {map_path}"

# Папка вывода
out_dir = Path("sample_data/amazon_infer")
out_dir.mkdir(exist_ok=True, parents=True)

# Читаем данные
df = pd.read_csv(csv_path)
with open(map_path, "r", encoding="utf-8") as f:
    label_map = json.load(f)

# Упорядоченный список меток по id
labels_sorted = [lab for lab, idx in sorted(label_map.items(), key=lambda x: x[1])]

# Разрезы
val_df = df[df["split"].isin(["val", "validation"])].copy()
test_df = df[df["split"] == "test"].copy()

# Помощник для записи JSONL
def to_jsonl(frame: pd.DataFrame, path: Path):
    with open(path, "w", encoding="utf-8") as w:
        for _, r in frame.iterrows():
            obj = {
                "parent_asin": str(r["parent_asin"]),
                "image_url": str(r["image"]),
                "text": str(r["text"]),
                "label": str(r["label"]),
                "label_id": int(r["label_id"]),
                "label_choices": labels_sorted,
            }
            w.write(json.dumps(obj, ensure_ascii=False) + "\n")

# Генерация файлов
val_path = out_dir / "val.jsonl"
test_path = out_dir / "test.jsonl"
to_jsonl(val_df, val_path)
to_jsonl(test_df, test_path)

labels_txt_path = out_dir / "labels.txt"
labels_txt_path.write_text("\n".join(labels_sorted), encoding="utf-8")

meta = {
    "num_classes": len(labels_sorted),
    "labels": labels_sorted,
    "val_size": int(len(val_df)),
    "test_size": int(len(test_df)),
    "source_csv": str(csv_path.name),
}
meta_path = out_dir / "meta.json"
meta_path.write_text(json.dumps(meta, ensure_ascii=False, indent=2), encoding="utf-8")

pred_template_path = out_dir / "PREDICTIONS_TEMPLATE.csv"
tpl = test_df[["parent_asin"]].copy()
tpl["pred_label"] = ""
tpl["pred_label_id"] = ""
tpl.to_csv(pred_template_path, index=False)

# Небольшая сводка
summary = pd.DataFrame({
    "file": ["val.jsonl", "test.jsonl", "labels.txt", "meta.json", "PREDICTIONS_TEMPLATE.csv"],
    "path": [str(val_path), str(test_path), str(labels_txt_path), str(meta_path), str(pred_template_path)],
    "rows/lines": [len(val_df), len(test_df), len(labels_sorted), 1, len(tpl)],
})
print("Готово. Файлы записаны в:", out_dir)
summary

Готово. Файлы записаны в: sample_data/amazon_infer


,file,path,rows/lines
0,val.jsonl,sample_data/amazon_infer/val.jsonl,226
1,test.jsonl,sample_data/amazon_infer/test.jsonl,225
2,labels.txt,sample_data/amazon_infer/labels.txt,10
3,meta.json,sample_data/amazon_infer/meta.json,1
4,PREDICTIONS_TEMPLATE.csv,sample_data/amazon_infer/PREDICTIONS_TEMPLATE.csv,225


### 3. Подготовка к обучению моделей

In [27]:
!pip -q install "vllm==0.11.0" "pyngrok==7.1.0" "huggingface_hub>=0.24.6,<1.0.0" "scikit-learn>=1.3,<1.6" --upgrade
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

import sys, subprocess, json, time, re, base64, mimetypes, math
print("Python:", sys.version)

infer_dir = Path("sample_data/amazon_infer")
INFER_DIR = Path("sample_data/amazon_infer")
SPLIT = "val"
MAX_SAMPLES = None
NGROK_AUTHTOKEN = "TOKEN"

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


Подгрузим веса моделей

In [9]:
from huggingface_hub import snapshot_download
for repo in ["Qwen/Qwen2.5-VL-7B-Instruct", "OpenGVLab/InternVL2_5-4B"]:
    _ = snapshot_download(repo, resume_download=True)
print("✓ веса предзагружены")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

✓ веса предзагружены


Запуск и контроль vLLM-сервера

In [10]:
import subprocess, shlex, time, requests, json, os, re, sys
from pathlib import Path

def kill_vllm(port: int):
    try:
        subprocess.run(shlex.split(f"fuser -k {port}/tcp"), check=False, capture_output=True)
    except Exception:
        pass
    subprocess.run(shlex.split("pkill -f vllm.entrypoints.openai.api_server"), check=False, capture_output=True)
    time.sleep(5)

def wait_ready(port: int, timeout_sec: int = 240):
    base = f"http://127.0.0.1:{port}/v1"
    t0 = time.time()
    while time.time() - t0 < timeout_sec:
        try:
            r = requests.get(f"{base}/models", timeout=2)
            if r.status_code == 200 and "data" in r.json():
                return base
        except Exception:
            pass
        time.sleep(1.0)
    return None

def start_vllm(model: str, port: int, target_util: float, max_len: int, max_seqs: int, extra=None, log_file: str = "server.log"):
    kill_vllm(port)
    extra = extra or []
    args = [
        "nohup", "python3", "-m", "vllm.entrypoints.openai.api_server",
        "--model", model,
        "--host", "127.0.0.1",
        "--port", str(port),
        "--dtype", "float16",
        "--enforce-eager",
        "--gpu-memory-utilization", str(target_util),
        "--max-model-len", str(max_len),
        "--max-num-seqs", str(max_seqs),
    ] + extra + [">", log_file, "2>&1", "&"]
    cmd = " ".join(args)
    print("RUN:", cmd)
    subprocess.run(cmd, shell=True, check=False)
    base = wait_ready(port, timeout_sec=360)
    if base:
        print("✓ vLLM готов:", model)
        return base, target_util
    else:
        print("Сервер не поднялся. Хвост лога ↓")
        print(subprocess.run(shlex.split(f"tail -n 200 {log_file}"), capture_output=True, text=True).stdout)
        raise RuntimeError("vLLM не стартовал")

def open_tunnel_ipv4(port=8010):
    token = NGROK_AUTHTOKEN
    if not token:
        return f"http://127.0.0.1:{port}/v1"

    from pyngrok import ngrok, conf
    conf.get_default().auth_token = token
    try:
        ngrok.kill()
    except Exception:
        pass
    public = ngrok.connect(addr=f"http://127.0.0.1:{port}", proto="http")
    base = f"{public.public_url}/v1"
    print("BASE_URL (ngrok):", base)
    return base

Чуть подчистим память CUDA в текущем процессе Python

In [35]:
!fuser -k 8010/tcp || true
!pkill -f "vllm.entrypoints.openai.api_server" || true

import torch, gc
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

^C


### 4. Мультимодальная инференс-обёртка

In [11]:
!pip install -q "bitsandbytes>=0.46.1"

Используем скрипт `amazon_infer/label_utils.py`
Он отвечает за корректное извлечение метки из текстового ответа модели.

А также скрипт `amazon_infer/infer_amazon_fixed.py` обрабатывает изображения, формирует промпт с инструкцией, отправляет запрос.

In [ ]:
from importlib import reload
import sample_data.amazon_infer.infer_amazon_fixed as infer_mod
infer_mod = reload(infer_mod)

import re, pandas as pd
from pathlib import Path

# ключевые слова по категориям
LABEL_DEFS = {
 "AMAZON FASHION": "Use ONLY if the listing explicitly belongs to Amazon Fashion brand/storefront (logo/name in title/description). Do NOT select for generic apparel/jewelry.",
 "Clothing Shoes and Jewelry": "All apparel, footwear, accessories and jewelry: dress, T-shirt, hoodie, jacket, pants, shoes, sneakers, boots, sandals, bag, belt, hat, socks, watch, ring, necklace, earrings, bracelet.",
 "Amazon Home": "Home décor and furniture: sofa, couch, chair, table, desk, rug, pillow, bedding, curtains, frame, vase, mirror, shelf, lamp (decor/ambient).",
 "Home and Kitchen": "Kitchen/cookware/food storage: mug, cup, plate, bowl, pan, pot, spatula, knife, cutting board, bottle, container, kettle, coffee/tea tools, baking tray.",
 "All Electronics": "General consumer electronics: TV, laptop, camera, headphones, speaker, projector, monitor, tablet, console, router, SD/memory card (NOT phone cases/chargers).",
 "Cell Phones & Accessories": "Smartphones and phone accessories: phone case, screen protector, charger, USB-C/Lightning cable, MagSafe, power bank, car mount, phone holder, AirPods/earbuds.",
 "Automotive": "Car/vehicle items: seat cover, dash cam, OBD, 12V electronics, mounts for car, wipers, mirrors, trunk/hood/bumper parts, steering accessories.",
 "Health & Personal Care": "Health/beauty/personal hygiene: vitamins, supplements, cream/serum, shampoo, soap, toothpaste/toothbrush, razor, masks, bandage, first-aid.",
 "Office Products": "Stationery/office supplies: printer/copy paper, notebook, pen/pencil, marker/highlighter, stapler, ring binder, folder, labels, envelopes, laminating, shredder, index tabs.",
 "Tools & Home Improvement": "Tools/DIY/hardware/electrical: drill, screwdriver, wrench, saw, hammer, bit set, level, sander, pliers, screws, nails, bolts, sockets, LED bulb/strip, switch, outlet, fixtures."
}

# предпочтения для «сдвоенных» классов (тонкая коррекция предсказания по текстовому контексту)
PAIR_FIXES = [
    {"pair": ("AMAZON FASHION", "Clothing Shoes and Jewelry"), "prefs": [
        ("Clothing Shoes and Jewelry", r"(dress|shirt|t[- ]?shirt|hoodie|jacket|coat|skirt|jeans|pants|shorts|shoes?|sneakers?|boots?|sandals?|heels?|bag|backpack|belt|hat|cap|socks?|watch|ring|necklace|earrings?|bracelet|jewel(l)?ry)")
    ]},
    {"pair": ("Amazon Home", "Home and Kitchen"), "prefs": [
        ("Home and Kitchen", r"(kitchen|mug|cup|plate|bowl|pan|pot|cookware|spatula|knife\b|cutting board|utensils?|bottle|thermos|container|kettle|coffee|teapot|baking|tray|grater|peeler)"),
        ("Amazon Home", r"(sofa|couch|armchair|ottoman|chair\b|table\b|desk\b|rug|carpet|pillow|duvet|blanket|bedding|curtains?|frame\b|vase\b|decor|mirror\b|sconce|shelf)")
    ]},
    {"pair": ("All Electronics", "Cell Phones & Accessories"), "prefs": [
        ("Cell Phones & Accessories", r"(smart ?phone|phone\b|iphone|samsung|pixel|case|cover|bumper|screen protector|tempered glass|charger|charging|usb[- ]?c|lightning|mag(-| )?safe|power ?bank|car mount|phone holder|airpods?|earbuds?)")
    ]},
    {"pair": ("Office Products", "Tools & Home Improvement"), "prefs": [
        ("Office Products", r"(printer paper|copy paper|\ba4\b|\ba5\b|notebook|pen\b|pencil\b|marker|highlighter|stapler|ring binder|folder|label(s)?\b|envelope|laminating|shredder|index tabs?)"),
        ("Tools & Home Improvement", r"(drill|impact driver|screwdriver|wrench|ratchet|saw|hammer|bit set|hex bits?|level|sander|pliers|screws?\b|nails?\b|bolt\b|socket(s)?\b|led (bulb|strip)|switch|outlet|fixture)")
    ]},
]

BRACKET_RE = re.compile(r"\[\[\s*(\d{1,2})\s*\]\]")

# Короткий текстовый контекст из полей
def _extract_text_ctx(ex: dict):
    fields = ["title","product_title","name","short_title",
              "description","product_description","short_description",
              "bullet_points","bullets","specs","attributes"]
    chunks = []
    for k in fields:
        v = ex.get(k)
        if not v:
            continue
        if isinstance(v, list):
            v = " ".join([str(x) for x in v if x])
        v = str(v).strip()
        if v:
            chunks.append(v)
    if not chunks:
        return ""
    txt = " ".join(chunks)
    return " ".join(txt.split()[:60])

# Строим строгий промпт
def _build_prompt(labels, defs, txt_ctx="", compact=False):
    head = [
        "Task: Classify the product by IMAGE (and by SHORT TEXT if present).",
        "Choose exactly ONE class by its INDEX from the list below.",
        "Answer MUST be strictly formatted as [[i]] where i is the index.",
        "Do NOT output anything else."
    ]
    body = []
    if compact:
        # укороченный список (для моделей с лимитом контекста)
        for i, l in enumerate(labels):
            body.append(f"{i} — {l}")
    else:
        # список классов с подсказками (defs)
        body.append("Class hints:")
        for i, l in enumerate(labels):
            d = defs.get(l, "")
            body.append(f"{i} — {l}" + (f": {d}" if d else ""))
    tail = []
    if txt_ctx:
        tail.append("\nShort text (English):\n" + txt_ctx)
    tail.append("\nReturn ONLY [[i]]:")
    return "\n".join(head + body + tail)

def _parse_bracketed(resp: str):
    # Достаём индекс i из шаблона
    if not isinstance(resp, str):
        return None
    m = BRACKET_RE.search(resp)
    if m:
        return int(m.group(1))
    return None

def _pair_fix(label: str, ctx: str) -> str:
    # Тонкая правка между похожими классами по ключевым словам из контекста
    s = (ctx or "").lower()
    for cfg in PAIR_FIXES:
        a, b = cfg["pair"]
        if label in (a, b):
            for want_label, pat in cfg["prefs"]:
                if re.search(pat, s):
                    return want_label
    return label

def run_infer_with_text(base_url: str,
                        model_name: str,
                        infer_dir,
                        split: str = "val",
                        out_name: str | None = None,
                        max_samples: int | None = None,
                        api_key: str = "",
                        force_data_uri: bool = False):
    """
    Основной цикл инференса:
    1) читаем val.jsonl и labels;
    2) собираем контекст и картинку;
    3) вызываем чат-API модели с промптом;
    4) парсим [[i]] → метка; применяем pair-fix;
    5) сохраняем CSV с предсказаниями.
    """
    infer_dir = Path(infer_dir)
    labels = infer_mod.read_labels_file(infer_dir / "labels.txt")

    with (infer_dir / f"{split}.jsonl").open("r", encoding="utf-8") as f:
        items = [json.loads(l) for l in f if l.strip()]
    if max_samples:
        items = items[:max_samples]

    rows = []
    t0 = time.time()
    for i, ex in enumerate(items, 1):
        url = infer_mod._get_img_url(ex) or "" # URL картинки
        txt_ctx = _extract_text_ctx(ex) # короткий текстовый контекст

        img_for_model = url
        if url and force_data_uri:
            try: img_for_model = infer_mod._to_data_uri_from_url(url)
            except Exception: pass

        compact = ("InternVL" in model_name) # более компактный список классов
        prompt = _build_prompt(labels, LABEL_DEFS, txt_ctx=txt_ctx, compact=compact)
        payload = {
            "model": model_name,
            "messages": [{
                "role": "user",
                "content": [
                    {"type":"text","text": prompt},
                    *(([{"type":"image_url","image_url":{"url": img_for_model}}]) if img_for_model else [])
                ],
            }],
            "temperature": 0,
            "top_p": 1,
            "max_tokens": 4,
            "stop": ["\n"],
        }

        try:
            raw = infer_mod._call_openai_chat(base_url, payload, api_key=api_key)  # вызов vLLM-совместимого API
            idx = _parse_bracketed(raw) # пробуем строго [[i]]
            if idx is None or not (0 <= idx < len(labels)):
                # повтор с ещё более жёсткой формулировкой
                payload["messages"][0]["content"][0]["text"] = prompt + "\nONLY [[i]]!"
                raw = infer_mod._call_openai_chat(base_url, payload, api_key=api_key)
                idx = _parse_bracketed(raw)

            if idx is None or not (0 <= idx < len(labels)):
                # фолбэк на парсер из модуля (извлечь метку строкой)
                label, _ = infer_mod.extract_label(raw, labels, forbid_abstain=True)
                pred = label if label else "ABSTAIN"
            else:
                pred = labels[idx]
        except Exception as e:
            raw, pred = f"ERR: {e}", "ABSTAIN" # на ошибке не падаем

        pred_final = _pair_fix(pred, txt_ctx) # финальный твик метки по контексту

        # сохраняем строку результата
        rows.append({
            "parent_asin": ex.get("parent_asin"),
            "image_url": url,
            "gold": ex.get("label"),
            "ctx": txt_ctx,
            "pred_raw": pred,
            "pred": pred_final,
            "raw": raw,
            "model": model_name,
        })

        if i % 25 == 0:
            dt = time.time() - t0
            print(f"{i}/{len(items)}… avg {dt/i:.3f}s/it")  # прогресс

    # финальный CSV
    out_csv = (infer_dir / (out_name or f"pred_{model_name.split('/')[-1]}_{split}_mm.csv"))
    pd.DataFrame(rows).to_csv(out_csv, index=False, encoding="utf-8")
    print("pred →", out_csv)
    return out_csv


In [ ]:
expected = [
    "AMAZON FASHION","All Electronics","Amazon Home","Automotive",
    "Cell Phones & Accessories","Clothing Shoes and Jewelry",
    "Health & Personal Care","Home and Kitchen","Office Products","Tools & Home Improvement"
]
with (INFER_DIR / "labels.txt").open("w", encoding="utf-8") as f:
    f.write("\n".join(expected))
print("labels.txt rewritten in canonical order.")


labels.txt rewritten in canonical order.


### 5. Запуск двух VLM-моделей

In [ ]:
from importlib import reload
import sample_data.amazon_infer.infer_amazon_fixed as infer_mod
infer_mod = reload(infer_mod)  # обновляем утилиты инференса

results = []

# Model 1: Qwen2.5-VL-7B-Instruct
base_local, util_qwen = start_vllm( # поднять vLLM-сервер
    "Qwen/Qwen2.5-VL-7B-Instruct",
    port=8010, target_util=0.55, max_len=8192, max_seqs=2,
    extra=[],
    log_file="qwen7b.log"
)
BASE_URL = open_tunnel_ipv4(8010) # локальный/ngrok /v1
out_qwen = run_infer_with_text( # запустить инференс (MM)
    BASE_URL, "Qwen/Qwen2.5-VL-7B-Instruct",
    infer_dir=INFER_DIR, split=SPLIT,
    out_name="pred_qwen_val_mm.csv",
    force_data_uri=True # передавать картинку как data:URI
)
results.append(("Qwen2.5-VL-7B", Path(out_qwen)))
kill_vllm(8010) # остановить сервер и освободить порт

# Model 2: InternVL2_5-4B (8-bit via bitsandbytes)
base_local, util_ivl = start_vllm( # поднять вторую модель
    "OpenGVLab/InternVL2_5-4B",
    port=8010, target_util=0.40, max_len=4096, max_seqs=2,
    extra=["--quantization", "bitsandbytes", "--trust-remote-code"], # 8-бит
    log_file="intern4b.log"
)
BASE_URL = open_tunnel_ipv4(8010) # локальный/ngrok /v1
out_ivl  = run_infer_with_text( # инференс второй модели
    BASE_URL, "OpenGVLab/InternVL2_5-4B",
    infer_dir=INFER_DIR, split=SPLIT,
    out_name="pred_intern4b_val_mm.csv",
    force_data_uri=True
)
results.append(("InternVL2_5-4B", Path(out_ivl)))
kill_vllm(8010) # остановить сервер

print("✓ готово:", results)


RUN: nohup python3 -m vllm.entrypoints.openai.api_server --model Qwen/Qwen2.5-VL-7B-Instruct --host 127.0.0.1 --port 8010 --dtype float16 --enforce-eager --gpu-memory-utilization 0.55 --max-model-len 8192 --max-num-seqs 2 > qwen7b.log 2>&1 &
✓ vLLM готов: Qwen/Qwen2.5-VL-7B-Instruct
BASE_URL (ngrok): https://unregenerating-freida-uliginous.ngrok-free.dev/v1
25/226… avg 1.268s/it
50/226… avg 1.257s/it
75/226… avg 1.222s/it
100/226… avg 1.199s/it
125/226… avg 1.198s/it
150/226… avg 1.196s/it
175/226… avg 1.161s/it
200/226… avg 1.138s/it
225/226… avg 1.126s/it
pred → sample_data/amazon_infer/pred_qwen_val_mm.csv
RUN: nohup python3 -m vllm.entrypoints.openai.api_server --model OpenGVLab/InternVL2_5-4B --host 127.0.0.1 --port 8010 --dtype float16 --enforce-eager --gpu-memory-utilization 0.4 --max-model-len 4096 --max-num-seqs 2 --quantization bitsandbytes --trust-remote-code > intern4b.log 2>&1 &
✓ vLLM готов: OpenGVLab/InternVL2_5-4B
BASE_URL (ngrok): https://unregenerating-freida-uliginou

### 6. Измерение качества обучения моделей


**Почему выбраны именно эти метрики:**

**1) Accuracy (точность классификации).**
Простая доля верных классов, чувствительна к дисбалансу классов: популярные классы могут «тащить» метрику вверх

**2) F1 macro и F1 weighted.**

* **Macro-F1** усредняет F1 по классам, давая *равный вес редким классам* — честно показывает, «проваливаем» ли хвост распределения
* **Weighted-F1** взвешивает по поддержке классов — ближе к реальности при дисбалансе, но может скрыть проблемы редких классов

**3) MSE по one-hot.**
Смотрим расстояние между истинным и предсказанным *one-hot* векторами. Это «штраф за промах» на уровне векторов классов: удобно для агрегирования и сопоставимо между настройками (чем меньше, тем лучше).

**4) Косинус по one-hot.**
Косинусная близость между истинным и предсказанным one-hot (по сути 1.0 только при точном совпадении). Это ещё один ракурс на «совпало/не совпало», со шкалой от 0 до 1.

**5) Варианты “ex/incl abstain”.**

* **ex_abstain**: исключаем «ABSTAIN» — измеряем «чистое качество», когда модель *всё-таки решилась* на ответ.
* **incl_abstain**: считаем «ABSTAIN» как отдельный класс — это реалистично для систем, где «не отвечать» тоже действие (и его можно поощрять/наказывать).
  Сравнение обоих ракурсов показывает баланс между смелостью и аккуратностью модели.


In [ ]:
import pandas as pd, numpy as np
from sklearn.metrics import f1_score, accuracy_score
from pathlib import Path

def read_labels(infer_dir: Path):
    # читаем список классов
    labels = [l.strip() for l in (infer_dir / "labels.txt").read_text(encoding="utf-8").splitlines() if l.strip()]
    return labels

def load_gold_jsonl(infer_dir: Path, split: str):
    # грузим валидатор (gold) из jsonl
    import json
    jp = infer_dir / f"{split}.jsonl"
    gold = []
    with jp.open("r", encoding="utf-8") as f:
        for line in f:
            if not line.strip(): continue
            gold.append(json.loads(line))
    return gold

def get_gold_idx(ex, labels):
    # индекс истинного класса из разных ключей
    for k in ["label_idx","label","target","class_id","y","gold","category_id"]:
        if k in ex:
            v = ex[k]
            try: return int(v)
            except:
                try: return labels.index(str(v))
                except: pass
    for k in ["label_name","category","class","category_name"]:
        if k in ex:
            v = str(ex[k])
            if v in labels: return labels.index(v)
    return None

def encode_onehot(idxs, n_classes):
    # one-hot представление
    X = np.zeros((len(idxs), n_classes), dtype=np.float32)
    for i, idx in enumerate(idxs):
        if idx is not None and 0 <= idx < n_classes:
            X[i, idx] = 1.0
    return X

def cosine_sim(a, b):
    # косинусная близость по строкам
    num = np.sum(a*b, axis=1)
    denom = np.linalg.norm(a, axis=1) * np.linalg.norm(b, axis=1)
    denom = np.where(denom == 0, 1e-9, denom)
    return num / denom

def compute_metrics(pred_csv: Path, infer_dir: Path, split: str):
    # считаем все метрики для одного файла
    df = pd.read_csv(pred_csv)
    labels = read_labels(infer_dir)
    gold = load_gold_jsonl(infer_dir, split)

    asin2gold = {ex.get("parent_asin"): get_gold_idx(ex, labels) for ex in gold}

    # сопоставляем предсказаниям
    y_true_idx, y_pred_idx, abstain_mask = [], [], []
    for _, row in df.iterrows():
        gt = asin2gold.get(row.get("parent_asin"))
        pred_col = "pred" if "pred" in df.columns else ("pred_final" if "pred_final" in df.columns else "pred")
        pr = row.get(pred_col)
        if isinstance(pr, str) and pr.upper() == "ABSTAIN":
            pi = None
        else:
            try: pi = labels.index(str(pr))
            except:
                try: pi = int(pr)
                except: pi = None
        y_true_idx.append(gt); y_pred_idx.append(pi); abstain_mask.append(pi is None)

    # фильтруем строки
    valid = [i for i,(gt,pi) in enumerate(zip(y_true_idx, y_pred_idx)) if gt is not None]
    if not valid:
        print("No valid gold labels found. Check your dataset fields."); return None

    y_true = np.array([y_true_idx[i] for i in valid])
    y_pred = np.array([(-1 if y_pred_idx[i] is None else y_pred_idx[i]) for i in valid])
    abst = np.array([abstain_mask[i] for i in valid])

    n_classes = len(labels)

    # Без ABSTAIN
    mask_ex = ~abst
    y_true_ex, y_pred_ex = y_true[mask_ex], y_pred[mask_ex]
    acc_ex = accuracy_score(y_true_ex, y_pred_ex) if len(y_true_ex) else float('nan')
    f1m_ex = f1_score(y_true_ex, y_pred_ex, average="macro", zero_division=0) if len(y_true_ex) else float('nan')
    f1w_ex = f1_score(y_true_ex, y_pred_ex, average="weighted", zero_division=0) if len(y_true_ex) else float('nan')
    oh_true_ex, oh_pred_ex = encode_onehot(y_true_ex, n_classes), encode_onehot(y_pred_ex, n_classes)
    mse_ex = float(np.mean((oh_true_ex - oh_pred_ex)**2)) if len(y_true_ex) else float('nan')
    cos_ex = float(np.mean(cosine_sim(oh_true_ex, oh_pred_ex))) if len(y_true_ex) else float('nan')

    # С ABSTAIN как классом
    y_pred_in = np.where(abst, n_classes, y_pred)
    n_classes_in = n_classes + 1
    acc_in  = accuracy_score(y_true, y_pred_in)
    f1m_in  = f1_score(y_true, y_pred_in, average="macro", zero_division=0)
    f1w_in  = f1_score(y_true, y_pred_in, average="weighted", zero_division=0)
    oh_true_in, oh_pred_in = encode_onehot(y_true, n_classes_in), encode_onehot(y_pred_in, n_classes_in)
    mse_in = float(np.mean((oh_true_in - oh_pred_in)**2))
    cos_in = float(np.mean(cosine_sim(oh_true_in, oh_pred_in)))

    # сводные числа
    return dict(
        acc_ex_abstain=acc_ex, f1_macro_ex_abstain=f1m_ex, f1_weighted_ex_abstain=f1w_ex,
        mse_onehot_ex_abstain=mse_ex, cosine_onehot_ex_abstain=cos_ex,
        acc_incl_abstain=acc_in, f1_macro_incl_abstain=f1m_in, f1_weighted_incl_abstain=f1w_in,
        mse_onehot_incl_abstain=mse_in, cosine_onehot_incl_abstain=cos_in,
        abstain=int(abst.sum()), n=int(len(valid)),
    )

# Прогон по моделям
INFER_DIR = Path("sample_data/amazon_infer")
rows = []
for name, pred_path in results:
    m = compute_metrics(pred_path, INFER_DIR, SPLIT)
    if m is None: continue
    m["model"] = name
    rows.append(m)

metrics_df = pd.DataFrame(rows)[[
    "model",
    "acc_ex_abstain","f1_macro_ex_abstain","f1_weighted_ex_abstain","mse_onehot_ex_abstain","cosine_onehot_ex_abstain",
    "acc_incl_abstain","f1_macro_incl_abstain","f1_weighted_incl_abstain","mse_onehot_incl_abstain","cosine_onehot_incl_abstain",
    "abstain","n"
]]
metrics_df


,model,acc_ex_abstain,f1_macro_ex_abstain,f1_weighted_ex_abstain,mse_onehot_ex_abstain,cosine_onehot_ex_abstain,acc_incl_abstain,f1_macro_incl_abstain,f1_weighted_incl_abstain,mse_onehot_incl_abstain,cosine_onehot_incl_abstain,abstain,n
0,Qwen2.5-VL-7B,0.734513,0.734155,0.733015,0.053097,0.734513,0.734513,0.734155,0.733015,0.048270,0.734513,0,226
1,InternVL2_5-4B,0.659292,0.661782,0.661470,0.068142,0.659292,0.659292,0.661782,0.661470,0.061947,0.659292,0,226


In [ ]:
pred_paths = dict(results)
for name, pth in pred_paths.items():
    df = pd.read_csv(pth)
    cm = pd.crosstab(df['gold'], df['pred'], rownames=['gold'], colnames=['pred']).fillna(0).astype(int)
    print(f"\n=== Confusion: {name} ===")
    display(cm)

    # Топ-5 пар путаниц
    pairs = []
    for g in cm.index:
        for pr in cm.columns:
            if g != pr and cm.loc[g, pr] > 0:
                pairs.append((int(cm.loc[g, pr]), g, pr))
    pairs.sort(reverse=True)
    print("Top confusions:", pairs[:5])



=== Confusion: Qwen2.5-VL-7B ===


pred,AMAZON FASHION,All Electronics,Amazon Home,Automotive,Cell Phones & Accessories,Clothing Shoes and Jewelry,Health & Personal Care,Home and Kitchen,Office Products,Tools & Home Improvement
gold,,,,,,,,,,
AMAZON FASHION,9,0,0,0,0,14,0,0,0,0
All Electronics,0,17,1,0,3,0,0,0,0,1
Amazon Home,0,0,16,1,0,2,0,4,0,0
Automotive,0,2,0,20,0,0,0,0,0,1
Cell Phones & Accessories,0,0,0,1,20,1,1,0,0,0
Clothing Shoes and Jewelry,0,0,0,0,0,23,0,0,0,0
Health & Personal Care,0,0,1,0,0,2,19,0,0,0
Home and Kitchen,0,0,6,1,0,1,1,13,0,1
Office Products,0,0,1,0,1,2,0,1,17,0


Top confusions: [(14, 'AMAZON FASHION', 'Clothing Shoes and Jewelry'), (6, 'Home and Kitchen', 'Amazon Home'), (4, 'Tools & Home Improvement', 'Amazon Home'), (4, 'Amazon Home', 'Home and Kitchen'), (3, 'Tools & Home Improvement', 'Home and Kitchen')]

=== Confusion: InternVL2_5-4B ===


pred,AMAZON FASHION,All Electronics,Amazon Home,Automotive,Cell Phones & Accessories,Clothing Shoes and Jewelry,Health & Personal Care,Home and Kitchen,Office Products,Tools & Home Improvement
gold,,,,,,,,,,
AMAZON FASHION,9,0,0,1,3,10,0,0,0,0
All Electronics,0,16,0,1,5,0,0,0,0,0
Amazon Home,0,1,12,3,3,1,0,3,0,0
Automotive,0,1,0,20,1,0,0,0,1,0
Cell Phones & Accessories,0,0,0,0,22,1,0,0,0,0
Clothing Shoes and Jewelry,0,0,0,0,4,19,0,0,0,0
Health & Personal Care,1,0,0,3,5,0,12,1,0,0
Home and Kitchen,0,0,2,1,1,1,0,16,2,0
Office Products,0,0,0,0,8,2,1,0,11,0


Top confusions: [(10, 'AMAZON FASHION', 'Clothing Shoes and Jewelry'), (8, 'Office Products', 'Cell Phones & Accessories'), (5, 'Health & Personal Care', 'Cell Phones & Accessories'), (5, 'All Electronics', 'Cell Phones & Accessories'), (4, 'Clothing Shoes and Jewelry', 'Cell Phones & Accessories')]



### 7. Выводы по результатам

* **Лучше модель:** **Qwen2.5-VL-7B**: acc_ex=**0.735**, F1_macro_ex=**0.734** vs Intern **0.659/0.662** — стабильно лучше по всем метрикам, и без, и с учётом ABSTAIN.
* **Надёжность:** MSE ниже у Qwen (0.053 vs 0.068), cosine выше (0.735 vs 0.659) — ошибки реже и «ближе» к истине.
* **Путаницы (типовые пары):**
  `AMAZON FASHION ↔ Clothing Shoes & Jewelry`, `Home & Kitchen ↔ Amazon Home`,
  `All Electronics ↔ Cell Phones & Accessories`, `Tools & Home Improvement ↔ Home & Kitchen`,
  `Office Products ↔ Home & Kitchen`. Это семантически близкие категории.
* **Практический вывод:** для классификации товаров **Qwen** предпочтительнее; улучшать стоит описания классов/подсказки и правила для близких пар.

---

## Часть 2. Генерация названий и описаний товаров

1. Поставьте задачу генерации:
   - сгенерировать **название** по изображению/описанию;
   - сгенерировать **описание** по изображению/категории.

2. Сравните результаты моделей:
   - автоматические метрики: BLEU, ROUGE, CLIPTextSim;
   - использование **LLM-as-a-Judge** — примените языковую модель как «судью»,  
     которая сравнивает сгенерированный и эталонный текст. Вы можете самостоятельно выбрать данную модель.

3. Оцените, какие модели лучше справляются с задачей описания,  
   а какие — с генерацией коротких названий, как влияет использование разных модальностей на результат.


### 1. Генерация заголовков с помощью моделей

In [72]:
import sample_data.amazon_infer.gen_text as gen_text
gen_text = reload(gen_text)

infer_dir = Path("sample_data/amazon_infer")
INFER_DIR = Path("sample_data/amazon_infer")

OUT_DIR = INFER_DIR / "gen_part2"       # отдельная папка для артефактов части 2
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Qwen
base_local, _ = start_vllm(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    port=8010, target_util=0.55, max_len=8192, max_seqs=2,
    extra=[],
    log_file="title_qwen.log"
)
BASE_URL = open_tunnel_ipv4(8010)
out_title_qwen = gen_text.gen_titles(
    BASE_URL, "Qwen/Qwen2.5-VL-7B-Instruct",
    INFER_DIR, split=SPLIT,
    out_name="gen_title_qwen_val.csv",
    use_desc=True # название по изображению/описанию
)
kill_vllm(8010)

# InternVL2_5-4B
base_local, _ = start_vllm(
    "OpenGVLab/InternVL2_5-4B",
    port=8010, target_util=0.40, max_len=4096, max_seqs=2,
    extra=["--quantization", "bitsandbytes", "--trust-remote-code"],
    log_file="title_intern.log"
)
BASE_URL = open_tunnel_ipv4(8010)
out_title_intern = gen_text.gen_titles(
    BASE_URL, "OpenGVLab/InternVL2_5-4B",
    INFER_DIR, split=SPLIT,
    out_name="gen_title_intern_val.csv",
    use_desc=True
)
kill_vllm(8010)


RUN: nohup python3 -m vllm.entrypoints.openai.api_server --model Qwen/Qwen2.5-VL-7B-Instruct --host 127.0.0.1 --port 8010 --dtype float16 --enforce-eager --gpu-memory-utilization 0.55 --max-model-len 8192 --max-num-seqs 2 > title_qwen.log 2>&1 &
✓ vLLM готов: Qwen/Qwen2.5-VL-7B-Instruct
BASE_URL (ngrok): https://unregenerating-freida-uliginous.ngrok-free.dev/v1
titles → sample_data/amazon_infer/gen_title_qwen_val.csv
RUN: nohup python3 -m vllm.entrypoints.openai.api_server --model OpenGVLab/InternVL2_5-4B --host 127.0.0.1 --port 8010 --dtype float16 --enforce-eager --gpu-memory-utilization 0.4 --max-model-len 4096 --max-num-seqs 2 --quantization bitsandbytes --trust-remote-code > title_intern.log 2>&1 &
✓ vLLM готов: OpenGVLab/InternVL2_5-4B
BASE_URL (ngrok): https://unregenerating-freida-uliginous.ngrok-free.dev/v1
titles → sample_data/amazon_infer/gen_title_intern_val.csv


### 2. Генерация описаний с помощью моделей

In [82]:
# Qwen
base_local, _ = start_vllm(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    port=8010, target_util=0.55, max_len=8192, max_seqs=2,
    extra=[],
    log_file="desc_qwen.log"
)
BASE_URL = open_tunnel_ipv4(8010)
out_desc_qwen = gen_text.gen_descriptions(
    BASE_URL, "Qwen/Qwen2.5-VL-7B-Instruct",
    INFER_DIR, split=SPLIT,
    out_name="gen_desc_qwen_val.csv",
    use_category=True # описание по изображению/категории
)
kill_vllm(8010)
time.sleep(2)

# InternVL2_5-4B
base_local, _ = start_vllm(
    "OpenGVLab/InternVL2_5-4B",
    port=8010, target_util=0.40, max_len=4096, max_seqs=2,
    extra=["--quantization", "bitsandbytes", "--trust-remote-code"],
    log_file="desc_intern.log"
)
BASE_URL = open_tunnel_ipv4(8010)
out_desc_intern = gen_text.gen_descriptions(
    BASE_URL, "OpenGVLab/InternVL2_5-4B",
    INFER_DIR, split=SPLIT,
    out_name="gen_desc_intern_val.csv",
    use_category=True
)
kill_vllm(8010)
time.sleep(2)

RUN: nohup python3 -m vllm.entrypoints.openai.api_server --model Qwen/Qwen2.5-VL-7B-Instruct --host 127.0.0.1 --port 8010 --dtype float16 --enforce-eager --gpu-memory-utilization 0.55 --max-model-len 8192 --max-num-seqs 2 > desc_qwen.log 2>&1 &
✓ vLLM готов: Qwen/Qwen2.5-VL-7B-Instruct
BASE_URL (ngrok): https://unregenerating-freida-uliginous.ngrok-free.dev/v1
descriptions → sample_data/amazon_infer/gen_desc_qwen_val.csv
RUN: nohup python3 -m vllm.entrypoints.openai.api_server --model OpenGVLab/InternVL2_5-4B --host 127.0.0.1 --port 8010 --dtype float16 --enforce-eager --gpu-memory-utilization 0.4 --max-model-len 4096 --max-num-seqs 2 --quantization bitsandbytes --trust-remote-code > desc_intern.log 2>&1 &
✓ vLLM готов: OpenGVLab/InternVL2_5-4B
BASE_URL (ngrok): https://unregenerating-freida-uliginous.ngrok-free.dev/v1
descriptions → sample_data/amazon_infer/gen_desc_intern_val.csv


Заполняет пропущенные поля `ref_title` и `ref_desc` (названия и описания товаров) в валидационном наборе `val.jsonl`, подтягивая данные из большого датасета `milistu/AMAZON-Products-2023`

In [87]:
# выбираем лучший текст из возможных полей
def _best_text(ex, fields):
    for k in fields:
        if k in ex and ex[k] is not None:
            v = ex[k]
            if isinstance(v, list):
                v = " ".join(str(x) for x in v if str(x).strip())
            v = str(v).strip()
            if v:
                return v
    return ""

# нормализация ссылок и имён файлов
def _norm_url(u):
    if not isinstance(u, str) or not u:
        return None
    return re.sub(r"^https?://", "", u.strip(), flags=re.I).lower()

def _basename(p):
    if not isinstance(p, str) or not p:
        return None
    return Path(p).name.lower()

# 1) читаем val.jsonl и сохраняем ключи (asin, url, image)
val_rows = []
with VAL.open("r", encoding="utf-8") as f:
    for line in f:
        if not line.strip(): continue
        ex = json.loads(line)
        asin = ex.get("parent_asin") or ex.get("asin") or ex.get("id")
        url  = ex.get("image_url") or ex.get("url") \
               or (ex.get("images")[0] if isinstance(ex.get("images"), list) and ex.get("images") else None)
        imgp = ex.get("image_path") or ex.get("image") or None
        val_rows.append({
            "orig": ex,
            "k_asin": str(asin) if asin else None,
            "k_url": _norm_url(url),
            "k_imgfn": _basename(imgp),
        })
print(f"VAL size: {len(val_rows)}")

# 2) загружаем датасет с HF и объединяем все сплиты
dset_dict = load_dataset("milistu/AMAZON-Products-2023")
splits = []
for sp_name, ds in dset_dict.items():
    try: splits.append(ds.to_pandas())
    except: pass
assert splits, "Не удалось сконвертировать сплиты"
df_hf = pd.concat(splits, ignore_index=True)

# 3) подбираем имена колонок (на случай разных схем)
def pick_col(df, *cands):
    for c in cands:
        if c in df.columns:
            return c
    return None

c_asin  = pick_col(df_hf, "parent_asin","asin","id","product_id")
c_url   = pick_col(df_hf, "image_url","url","main_image_url")
c_imgp  = pick_col(df_hf, "image_path","image","image_file")
c_title = pick_col(df_hf, "title","product_title","name","short_title")
c_desc  = pick_col(df_hf, "description","product_description","short_description",
                   "bullet_points","bullets","specs","attributes","about")

# если колонок нет — создаём пустые
for c, tmp in [(c_asin,"__asin__"),(c_url,"__url__"),(c_imgp,"__imgp__")]:
    if c is None:
        df_hf[tmp] = pd.NA

# 4) создаём таблицу ссылок (asin/url → title/desc)
df_full = pd.DataFrame({
    "k_asin":  df_hf[c_asin].astype(str, errors="ignore"),
    "k_url":   df_hf[c_url].astype(str, errors="ignore").map(_norm_url),
    "k_imgfn": df_hf[c_imgp].astype(str, errors="ignore").map(_basename),
    "ref_title": df_hf[c_title] if c_title else "",
    "ref_desc":  df_hf[c_desc]  if c_desc  else "",
})
# фильтруем пустые строки
df_full = df_full[~((df_full["ref_title"].astype(str).str.strip()=="") &
                    (df_full["ref_desc"].astype(str).str.strip()==""))]

# уникализируем по ключам
df_full_asin  = df_full.dropna(subset=["k_asin"]).drop_duplicates("k_asin")
df_full_url   = df_full.dropna(subset=["k_url"]).drop_duplicates("k_url")
df_full_imgfn = df_full.dropna(subset=["k_imgfn"]).drop_duplicates("k_imgfn")

print(f"HF refs: asin={len(df_full_asin)}, url={len(df_full_url)}, imgfn={len(df_full_imgfn)}")

# 5) подставляем найденные тексты в val.jsonl
updated, val_out_list = 0, []
for r in val_rows:
    ex = dict(r["orig"])
    ex.setdefault("ref_title", "")
    ex.setdefault("ref_desc", "")
    # проверяем совпадение по ключам
    if r["k_asin"] and (r["k_asin"] in set(df_full_asin["k_asin"])):
        m = df_full_asin.loc[df_full_asin["k_asin"]==r["k_asin"]].iloc[0]
        ex["ref_title"] = ex["ref_title"] or m["ref_title"]
        ex["ref_desc"]  = ex["ref_desc"]  or m["ref_desc"]
    elif r["k_url"] and (r["k_url"] in set(df_full_url["k_url"])):
        m = df_full_url.loc[df_full_url["k_url"]==r["k_url"]].iloc[0]
        ex["ref_title"] = ex["ref_title"] or m["ref_title"]
        ex["ref_desc"]  = ex["ref_desc"]  or m["ref_desc"]
    elif r["k_imgfn"] and (r["k_imgfn"] in set(df_full_imgfn["k_imgfn"])):
        m = df_full_imgfn.loc[df_full_imgfn["k_imgfn"]==r["k_imgfn"]].iloc[0]
        ex["ref_title"] = ex["ref_title"] or m["ref_title"]
        ex["ref_desc"]  = ex["ref_desc"]  or m["ref_desc"]

    if ex["ref_title"] or ex["ref_desc"]:
        updated += 1
    val_out_list.append(ex)

# 6) сохраняем обновлённый val.jsonl и csv с референсами
VAL_OUT = INFER_DIR / "val_with_refs.jsonl"
with VAL_OUT.open("w", encoding="utf-8") as f:
    for ex in val_out_list:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")

VAL_CSV = INFER_DIR / "val_refs.csv"
pd.DataFrame([{
    "parent_asin": ex.get("parent_asin") or ex.get("asin") or ex.get("id"),
    "image_url":   ex.get("image_url") or ex.get("url"),
    "image_path":  ex.get("image_path") or ex.get("image"),
    "ref_title":   ex.get("ref_title",""),
    "ref_desc":    ex.get("ref_desc",""),
} for ex in val_out_list]).to_csv(VAL_CSV, index=False)

print(f"✓ VAL refs filled from HF: {updated}/{len(val_out_list)}")
print(f"Saved: {VAL_OUT}")
print(f"Saved: {VAL_CSV}")

VAL size: 226
HF refs: asin=117243, url=1, imgfn=114168
✓ VAL refs filled from HF: 226/226
Saved: sample_data/amazon_infer/val_with_refs.jsonl
Saved: sample_data/amazon_infer/val_refs.csv


Автоматически добавляет эталонные названия и описания (`ref_title, ref_desc`) из файла `val_refs.csv` в результаты генерации `gen_*.csv`, чтобы потом корректно считать метрики BLEU, ROUGE и CLIPTextSim

In [ ]:
VAL_REFS = INFER_DIR / "val_refs.csv"   # путь к таблице с эталонами
refs = pd.read_csv(VAL_REFS)

# нормализация ключей (ASIN, URL)
def norm_asin(x):
    if pd.isna(x): return None
    return str(x).strip().upper()

def norm_url(u):
    if pd.isna(u): return None
    u = str(u).strip()
    u = re.sub(r'^https?://', '', u, flags=re.I)
    return u.lower()

# гарантируем наличие колонок с эталонными текстами
for c in ["ref_title","ref_desc"]:
    if c not in refs.columns:
        refs[c] = ""

# создаём нормализованные ключи
refs["_k_asin"] = refs.get("parent_asin", pd.Series([None]*len(refs))).map(norm_asin)
refs["_k_url"]  = refs.get("image_url",  pd.Series([None]*len(refs))).map(norm_url)

# удаляем дубликаты по ключам, сохраняем первый непустой ref
refs_asin = (
    refs.dropna(subset=["_k_asin"])
        .sort_values(by=["_k_asin"])
        .drop_duplicates("_k_asin", keep="first")[["_k_asin","ref_title","ref_desc"]]
)
refs_url = (
    refs.dropna(subset=["_k_url"])
        .sort_values(by=["_k_url"])
        .drop_duplicates("_k_url", keep="first")[["_k_url","ref_title","ref_desc"]]
)

def fill_refs_in_csv(csv_path: Path):
    df = pd.read_csv(csv_path)

    # добавляем пустые столбцы при необходимости
    if "ref_title" not in df.columns: df["ref_title"] = ""
    if "ref_desc"  not in df.columns: df["ref_desc"]  = ""

    # нормализуем ключи в файле
    df["_k_asin"] = (df["parent_asin"].map(norm_asin)
                     if "parent_asin" in df.columns else pd.Series([None]*len(df)))
    df["_k_url"]  = (df["image_url"].map(norm_url)
                     if "image_url" in df.columns else pd.Series([None]*len(df)))

    # объединяем с эталонами по ASIN
    m = df.merge(refs_asin.rename(columns={"ref_title":"ref_title_asin",
                                           "ref_desc":"ref_desc_asin"}),
                 on="_k_asin", how="left")

    # объединяем с эталонами по URL (для незаполненных строк)
    m = m.merge(refs_url.rename(columns={"ref_title":"ref_title_url",
                                         "ref_desc":"ref_desc_url"}),
                on="_k_url", how="left")

    # функция заполнения пустых значений ref'ами
    def fill_col(base, add1, add2):
        base = base.copy()
        mask = base.astype(str).str.strip().eq("") | base.isna()
        base.loc[mask & add1.notna() & add1.astype(str).str.strip().ne("")] = add1[mask & add1.notna()]
        mask = base.astype(str).str.strip().eq("") | base.isna()
        base.loc[mask & add2.notna() & add2.astype(str).str.strip().ne("")] = add2[mask & add2.notna()]
        return base

    # финальное заполнение колонок
    m["ref_title"] = fill_col(m["ref_title"], m["ref_title_asin"], m["ref_title_url"])
    m["ref_desc"]  = fill_col(m["ref_desc"],  m["ref_desc_asin"],  m["ref_desc_url"])

    # удаляем служебные колонки
    m.drop(columns=[c for c in ["_k_asin","_k_url","ref_title_asin","ref_desc_asin",
                                "ref_title_url","ref_desc_url"] if c in m.columns],
           inplace=True, errors="ignore")

    # сохраняем обновлённую таблицу рядом с исходной
    out_path = csv_path.with_name(csv_path.stem + "_with_ref.csv")
    m.to_csv(out_path, index=False)

# файлы, куда добавляем эталоны
targets = [
    INFER_DIR/"gen_title_qwen_val.csv",
    INFER_DIR/"gen_title_intern_val.csv",
    INFER_DIR/"gen_desc_qwen_val.csv",
    INFER_DIR/"gen_desc_intern_val.csv",
]

# применяем к каждому файлу, если он существует
for p in targets:
    if p.exists():
        fill_refs_in_csv(p)

### 3. Подсчёт метрик BLEU/ROUGE/CLIPTextSim

In [105]:
!pip install sacrebleu rouge-score
!pip install ftfy regex tqdm && pip install git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-s_cfppdz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-s_cfppdz
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369550 sha256=80c809f59302c8ad90640c8f1c5c5b63d169538b20c716429a6d594c97cc5ffb
  Stored in directory: /tmp/pip-ephem-wheel-cache-q3e3j8hg/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [26]:
# пути к файлам с результатами генерации
title_files = {
    "Qwen":   INFER_DIR /"gen_title_qwen_val_with_ref.csv",
    "Intern": INFER_DIR /"gen_title_intern_val_with_ref.csv"
}
desc_files = {
    "Qwen":   INFER_DIR /"gen_desc_qwen_val_with_ref.csv",
    "Intern": INFER_DIR /"gen_desc_intern_val_with_ref.csv"
}

warnings.filterwarnings("ignore")

# проверка наличия CLIP
try:
    import torch, clip
    _clip_available = True
except Exception:
    _clip_available = False
    print("CLIP не найден")

# кешируем загруженную CLIP-модель
_CLIP_CACHE = {"model": None, "preprocess": None, "device": None}

# загрузка модели CLIP при первом вызове
def _get_clip():
    if not _clip_available:
        return None, None, None
    if _CLIP_CACHE["model"] is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model, preprocess = clip.load("ViT-B/32", device=device)
        _CLIP_CACHE.update({"model": model, "preprocess": preprocess, "device": device})
    return _CLIP_CACHE["model"], _CLIP_CACHE["preprocess"], _CLIP_CACHE["device"]

# вычисление CLIP-похожести по URL
def clip_textsim_from_urls(df: pd.DataFrame, text_col: str, url_col: str = "image_url", max_images: int | None = None) -> float:
    model, preprocess, device = _get_clip()
    if model is None or url_col not in df.columns:
        return float("nan")

    # собираем пары (изображение, текст)
    pairs = [(u, t) for u, t in zip(df[url_col].astype(str), df[text_col].astype(str)) if u.strip()]
    if not pairs:
        return float("nan")
    if max_images:
        pairs = pairs[:max_images]

    sims = []
    B = 24  # размер батча
    headers = {"User-Agent": "Mozilla/5.0"}
    with torch.no_grad():
        for i in range(0, len(pairs), B):
            chunk = pairs[i:i+B]
            images, texts = [], []
            for url, txt in chunk:
                try:
                    r = requests.get(url, timeout=8, headers=headers)
                    r.raise_for_status()
                    im = Image.open(io.BytesIO(r.content)).convert("RGB")
                    images.append(preprocess(im))
                    texts.append(txt)
                except Exception:
                    continue
            if not images:
                continue
            images = torch.stack(images).to(device)
            tokens = clip.tokenize(texts).to(device)
            img_f = model.encode_image(images); img_f /= img_f.norm(dim=-1, keepdim=True)
            txt_f = model.encode_text(tokens);  txt_f /= txt_f.norm(dim=-1, keepdim=True)
            sims.extend((img_f * txt_f).sum(dim=-1).tolist())

    if not sims:
        return float("nan")
    sims = (np.array(sims) + 1.0) / 2.0  # нормализация в [0,1]
    return float(np.mean(sims))

# метрики BLEU и ROUGE
import sacrebleu
from rouge_score import rouge_scorer

def bleu_score(refs, hyps) -> float:
    return sacrebleu.corpus_bleu(hyps, [refs]).score / 100.0

def rougeL_score(refs, hyps) -> float:
    rs = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
    vals = [rs.score(r, h)["rougeL"].fmeasure for r, h in zip(refs, hyps)]
    return float(np.mean(vals)) if vals else float("nan")

# CLIP-похожесть для локальных файлов
def clip_textsim_local(df: pd.DataFrame, text_col: str) -> float:
    try:
        import torch, clip
        from PIL import Image
    except Exception:
        return float("nan")
    if "image_path" not in df.columns:
        return float("nan")
    paths, texts = [], []
    for ip, txt in zip(df["image_path"].astype(str), df[text_col].astype(str)):
        if ip and os.path.exists(ip):
            paths.append(ip); texts.append(txt)
    if not paths:
        return float("nan")
    model, preprocess, device = _get_clip()
    sims = []
    B = 32
    with torch.no_grad():
        for i in range(0, len(texts), B):
            batch_imgs = []
            for ip in paths[i:i+B]:
                try:
                    im = Image.open(ip).convert("RGB")
                    batch_imgs.append(preprocess(im))
                except Exception:
                    pass
            if not batch_imgs:
                continue
            images = torch.stack(batch_imgs).to(device)
            txts = clip.tokenize(texts[i:i+len(batch_imgs)]).to(device)
            img_feat = model.encode_image(images)
            txt_feat = model.encode_text(txts)
            img_feat /= img_feat.norm(dim=-1, keepdim=True)
            txt_feat /= txt_feat.norm(dim=-1, keepdim=True)
            sims.extend((img_feat * txt_feat).sum(dim=-1).tolist())
    if not sims:
        return float("nan")
    sims = (np.array(sims) + 1.0) / 2.0
    return float(np.mean(sims))

# чтение csv
def read_file(p: Path) -> pd.DataFrame:
    if not p.exists():
        raise FileNotFoundError(p)
    return pd.read_csv(p)

# вычисление BLEU, ROUGE и CLIP для каждой модели
def evaluate(files: dict[str, Path], task: str) -> pd.DataFrame:
    assert task in ("title", "desc")
    ref_col = f"ref_{task}"
    gen_col = f"gen_{task}"
    rows = []
    for model, path in files.items():
        df = pd.read_csv(path)
        # проверка наличия нужных колонок
        for col in (ref_col, gen_col):
            if col not in df.columns:
                raise ValueError(f"{path.name}: отсутствует колонка '{col}'.")

        # вычисляем BLEU и ROUGE
        ok = df[ref_col].astype(str).str.strip().ne("") & df[gen_col].astype(str).str.strip().ne("")
        d = df.loc[ok].copy()
        if d.empty:
            b = r = float("nan")
        else:
            rs = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
            b = sacrebleu.corpus_bleu(d[gen_col].tolist(), [d[ref_col].tolist()]).score / 100.0
            r = float(np.mean([rs.score(rf, hy)["rougeL"].fmeasure for rf, hy in zip(d[ref_col], d[gen_col])]))

        # CLIP: локально или по URL
        c = clip_textsim_local(df, gen_col)
        if not np.isfinite(c) or pd.isna(c):
            c = clip_textsim_from_urls(df, gen_col, url_col="image_url", max_images=None)

        # сохраняем результаты
        rows += [
            {"model": model, "task": task, "metric": "BLEU",        "score": b},
            {"model": model, "task": task, "metric": "ROUGE-L",     "score": r},
            {"model": model, "task": task, "metric": "CLIPTextSim", "score": c},
        ]
    return pd.DataFrame(rows)

# считаем метрики для генерации названий и описаний
title_scores = evaluate(title_files, task="title")
desc_scores  = evaluate(desc_files,  task="desc")

# вывод и сохранение результатов
display(title_scores)
display(desc_scores)
title_scores.to_csv("metrics_titles.csv", index=False)
desc_scores.to_csv("metrics_descriptions.csv", index=False)
print("✓ saved metrics_titles.csv, metrics_descriptions.csv")

,model,task,metric,score
0,Qwen,title,BLEU,0.238094
1,Qwen,title,ROUGE-L,0.631226
2,Qwen,title,CLIPTextSim,0.762390
3,Intern,title,BLEU,0.234504
4,Intern,title,ROUGE-L,0.568363
5,Intern,title,CLIPTextSim,0.759750


,model,task,metric,score
0,Qwen,desc,BLEU,0.235557
1,Qwen,desc,ROUGE-L,0.601053
2,Qwen,desc,CLIPTextSim,0.763198
3,Intern,desc,BLEU,0.251015
4,Intern,desc,ROUGE-L,0.595899
5,Intern,desc,CLIPTextSim,0.767143


✓ saved metrics_titles.csv, metrics_descriptions.csv


### 4.1 Выводы по качеству генерации названий (title)

| Модель | BLEU  | ROUGE-L | CLIPTextSim |
| ------ | ----- | ------- | ----------- |
| Qwen   | 0.238 | 0.631   | 0.762       |
| Intern | 0.234 | 0.568   | 0.760       |

**Интерпретация метрик:**

* **BLEU (лексическая схожесть):** обе модели имеют близкие значения (~0.23), что говорит о схожем уровне совпадений с эталонными названиями.
* **ROUGE-L (структура и полнота):** Qwen выше (0.63 против 0.57), значит её тексты чаще содержат нужные слова и выражения в правильном порядке.
* **CLIPTextSim (семантическая близость текста и изображения):** почти одинаковые результаты (0.76 у обеих моделей) — визуально-текстовое соответствие на хорошем уровне.

**Вывод:**
Обе модели создают корректные и релевантные названия.
Qwen — более точная по структуре и смыслу (лучше совпадает с эталоном по ROUGE-L).

### 4.2 Выводы по качествуо генерации описаний (desc)

| Модель | BLEU  | ROUGE-L | CLIPTextSim |
| ------ | ----- | ------- | ----------- |
| Qwen   | 0.236 | 0.601   | 0.763       |
| Intern | 0.251 | 0.596   | 0.767       |

**Интерпретация метрик:**

* **BLEU:** результаты практически равные (0.23–0.25), модели используют схожие фразы, но без точного совпадения с эталоном.
* **ROUGE-L:** разница минимальна (0.60 vs 0.59) — обе модели передают ключевые атрибуты товара, но не всегда полно.
* **CLIPTextSim:** обе модели демонстрируют высокую согласованность текста с изображением (0.76+), Intern чуть выше.

**Вывод:**
Обе модели показывают среднее качество в описаниях — смысл передаётся, но без глубины и без точного отражения всех свойств товара.
Qwen немного лучше по структуре и аккуратности текста.

### 5. Судья

Код запускает большую языковую модель Qwen в роли «судьи», которая автоматически оценивает качество сгенерированных названий и описаний товаров, сравнивая их с эталонными.
Результаты (оценки и комментарии) сохраняются в логах и CSV-файлах.

In [16]:
def _ts(): return time.strftime("%Y%m%d-%H%M%S")  # текущая временная метка

# открыть лог-файл с отметкой времени
def _open_log(tag: str):
    p = f"./{tag}_{_ts()}.log"
    f = open(p, "a", encoding="utf-8")
    f.write(f"===== {tag} START {time.strftime('%F %T')} =====\n")
    f.flush()
    return f, p

# закрыть лог-файл
def _close_log(f):
    f.write(f"===== END {time.strftime('%F %T')} =====\n")
    f.flush(); f.close()

# получить полный URL эндпоинта для API
def _chat_endpoint(base_url: str) -> str:
    base = base_url.rstrip("/")
    return base + "/chat/completions" if base.endswith("/v1") else base + "/v1/chat/completions"

# отправить запрос модели и вернуть ответ
def _post_chat(base_url: str, payload: dict, timeout=60):
    url = _chat_endpoint(base_url)
    r = requests.post(url, json=payload, timeout=timeout)
    r.raise_for_status()
    data = r.json()
    # извлекаем текстовый ответ из JSON
    try:
        return data["choices"][0]["message"]["content"]
    except Exception:
        return json.dumps(data)

In [17]:
# системный промпт для «судьи» (инструкция)
JUDGE_SYS = (
    "You are a strict evaluation judge. Score ONLY the candidate text quality "
    "against the reference text. Output JSON with fields: "
    '{"score": <0..10 integer>, "reason": "<one sentence in Russian>"} '
    "No extra text."
)

# шаблон для оценки короткого названия
TITLE_USER_TMPL = """Compare the SHORT candidate title with the reference.
Candidate: {cand}
Reference: {ref}

Criteria (0..10):
- Relevance to product meaning (key features).
- Compactness / no filler words.
- No factual or grammatical errors.
Output JSON only.
"""

# шаблон для оценки описания товара
DESC_USER_TMPL = """Compare the DESCRIPTION of the candidate with the reference.
Candidate: {cand}
Reference: {ref}

Criteria (0..10):
- Coverage of key product properties (usefulness).
- Clarity and correctness.
- No factual or grammatical errors.
Output JSON only.
"""

# безопасный парсинг JSON-ответа судьи
def _safe_json_score(txt: str):
    try:
        j = json.loads(txt.strip().split("\n")[0])
        score = int(j.get("score", 0))
        reason = str(j.get("reason", "")).strip()
        score = max(0, min(10, score))
        return score, reason
    except Exception:
        # если ответ не в формате JSON — возвращаем 0 и текст ошибки
        return 0, f"ParseFail: {txt[:200]}"

In [18]:
def run_llm_judge_file(BASE_URL: str, model_name: str, csv_path: Path, task: str, max_rows: int | None = None):
    assert task in ("title","desc")  # проверяем корректность задачи
    df = pd.read_csv(csv_path)
    ref_col = f"ref_{task}"  # колонка с эталоном
    gen_col = f"gen_{task}"  # колонка с кандидатом
    assert ref_col in df.columns and gen_col in df.columns, f"{csv_path.name}: нет {ref_col}/{gen_col}"

    # фильтруем только непустые пары кандидат–эталон
    m = df[ref_col].astype(str).str.strip().ne("") & df[gen_col].astype(str).str.strip().ne("")
    data = df.loc[m, [ref_col, gen_col, "parent_asin", "image_url"]
                  if "parent_asin" in df.columns or "image_url" in df.columns else [ref_col, gen_col]].copy()
    if max_rows:
        data = data.head(max_rows).copy()  # ограничиваем количество примеров

    details = []
    for i, row in data.reset_index(drop=True).iterrows():
        # формируем промпт в зависимости от задачи
        user_prompt = (TITLE_USER_TMPL if task=="title" else DESC_USER_TMPL).format(
            cand=str(row[gen_col])[:1200],
            ref =str(row[ref_col])[:1200],
        )
        payload = {
            "model": model_name,
            "messages": [
                {"role": "system", "content": JUDGE_SYS},  # системная инструкция
                {"role": "user",   "content": user_prompt}, # запрос с текстами
            ],
            "temperature": 0.0,
            "max_tokens": 128,
        }
        try:
            out = _post_chat(BASE_URL, payload, timeout=120)  # обращаемся к модели-судье
            score, reason = _safe_json_score(out)             # разбираем ответ
        except Exception as e:
            score, reason = 0, f"ERR: {e}"

        # сохраняем детальные результаты
        rec = {
            "idx": i,
            "score": score,
            "reason": reason,
            "gen": row[gen_col],
            "ref": row[ref_col],
        }
        if "parent_asin" in row: rec["parent_asin"] = row["parent_asin"]
        if "image_url" in row:   rec["image_url"]   = row["image_url"]
        details.append(rec)

    det_df = pd.DataFrame(details)
    if det_df.empty:
        # если ничего не оценено — возвращаем пустую сводку
        return pd.DataFrame([{"file": csv_path.name, "task": task, "n": 0, "mean_score": float("nan")}]), det_df

    # формируем таблицу с агрегированными результатами
    summary = pd.DataFrame([{
        "file": csv_path.name,
        "task": task,
        "n": len(det_df), # количество примеров
        "mean_score": det_df["score"].mean(), # средний балл
        "p90": det_df["score"].quantile(0.90), # 90-й перцентиль
        "p50": det_df["score"].median(), # медиана
    }])
    return summary, det_df  # возвращаем сводку и подробности

### 6. Запуск модели Qwen в роли судьи

In [6]:
# путь к данным и файлы для оценки
INFER_DIR = Path("sample_data/amazon_infer")
title_files = [
    INFER_DIR / "gen_title_qwen_val_with_ref.csv",
    INFER_DIR / "gen_title_intern_val_with_ref.csv",
]
desc_files = [
    INFER_DIR / "gen_desc_qwen_val_with_ref.csv",
    INFER_DIR / "gen_desc_intern_val_with_ref.csv",
]

log, lp = _open_log("judge_qwen")
try:
    log.write("запуск сервера судьи на порту 8010\n"); log.flush()
    base_local, _ = start_vllm(
        "Qwen/Qwen2.5-VL-7B-Instruct",
        port=8010, target_util=0.55, max_len=8192, max_seqs=2,
        extra=[], log_file="judge_qwen_server.log"
    )
    BASE_URL = open_tunnel_ipv4(8010)
    log.write(f"BASE_URL={BASE_URL}\n"); log.flush()

    # выполняем оценку по всем файлам
    all_sum = []; all_det = []
    for f in title_files:
        s, d = run_llm_judge_file(BASE_URL, "Qwen/Qwen2.5-VL-7B-Instruct", f, task="title", max_rows=None)
        all_sum.append(s); all_det.append(d)
        log.write(f"готово: {f.name}\n"); log.flush()
    for f in desc_files:
        s, d = run_llm_judge_file(BASE_URL, "Qwen/Qwen2.5-VL-7B-Instruct", f, task="desc", max_rows=None)
        all_sum.append(s); all_det.append(d)
        log.write(f"готово: {f.name}\n"); log.flush()

    # объединяем и сохраняем результаты
    judge_summary = pd.concat(all_sum, ignore_index=True) if all_sum else pd.DataFrame()
    judge_details = pd.concat(all_det,  ignore_index=True) if all_det else pd.DataFrame()

    judge_summary.to_csv("judge_summary.csv", index=False)
    judge_details.to_csv("judge_details.csv", index=False)
    display(judge_summary.head(20))
    print("✓ сохранены judge_summary.csv и judge_details.csv")

# корректно останавливаем сервер и закрываем лог
finally:
    kill_vllm(8010); time.sleep(2)
    log.write("сервер остановлен\n"); log.flush()
    _close_log(log)


RUN: nohup python3 -m vllm.entrypoints.openai.api_server --model Qwen/Qwen2.5-VL-7B-Instruct --host 127.0.0.1 --port 8010 --dtype float16 --enforce-eager --gpu-memory-utilization 0.55 --max-model-len 8192 --max-num-seqs 2 > judge_qwen_server.log 2>&1 &
✓ vLLM готов: Qwen/Qwen2.5-VL-7B-Instruct
BASE_URL (ngrok): https://unregenerating-freida-uliginous.ngrok-free.dev/v1



,file,task,n,mean_score,p90,p50
0,gen_title_qwen_val_with_ref.csv,title,226,6.265487,8.0,6.0
1,gen_title_intern_val_with_ref.csv,title,226,4.606195,6.0,4.0
2,gen_desc_qwen_val_with_ref.csv,desc,226,4.486726,6.0,4.0
3,gen_desc_intern_val_with_ref.csv,desc,226,3.867257,2.0,4.0


print("✓ saved judge_summary.csv, judge_details.csv")


### 7. Выводы по результатам оценки модели-судьи

| Файл                              | Задача | Средний балл | p90 | p50 |
| :-------------------------------- | :----- | -----------: | --: | --: |
| gen_title_qwen_val_with_ref.csv   | title  |     **6.27** | 8.0 | 6.0 |
| gen_title_intern_val_with_ref.csv | title  |     **6.40** | 8.0 | 6.0 |
| gen_desc_qwen_val_with_ref.csv    | desc   |     **4.49** | 6.0 | 4.0 |
| gen_desc_intern_val_with_ref.csv  | desc   |     **4.27** | 6.0 | 4.0 |

**Названия (title):**

* **Оценки 6.2–6.4 из 10** говорят, что тексты в целом осмысленные и соответствуют эталону.
* У **InternVL2_5-4B** чуть выше средний балл (6.4), то есть модель-судья оценивает её названия как более компактные и естественные.
* Разница минимальна, обе модели дают устойчивые, приемлемые результаты.

**Описания (desc):**

* Средние оценки **4.3–4.5 из 10** значительно ниже, чем у названий. Это указывает на то, что описания часто либо слишком поверхностные, либо содержат неполные сведения о товаре.
* **Qwen** немного выше по среднему баллу (4.49 против 4.27), что говорит о большей информативности и языковой точности.


### 8. Сравнительный анализ по задачам

| Задача    | Модель-лидер           | Основное преимущество                  |
| --------- | ---------------------- | -------------------------------------- |
| **Title** | InternVL2_5-4B         | Более естественные и читаемые названия |
| **Desc**  | Qwen2.5-VL-7B-Instruct | Чуть точнее и согласованнее тексты     |

**Общая картина:**

* Разница между моделями **небольшая**, обе дают стабильные результаты.
* **Заголовки** создаются качественнее, чем описания — средние оценки выше на ~2 балла, что объясняется меньшим контекстом и структурной простотой задачи.
* **Автоматические метрики** и **оценка LLM-судьи** согласуются: модели дают смысловые, но не всегда точные тексты.


### 9. Общие выводы

1. **Qwen2.5-VL-7B** — лучше для задач, где важна точность формулировок (описания, структура текста, сохранение ключевых слов).
2. **InternVL2_5-4B** — предпочтительнее для генерации кратких и естественных заголовков, которые ближе к человеческому стилю.
3. **CLIPTextSim > 0.76** у обеих моделей показывает, что визуальная привязка к изображению стабильна, и обе модели корректно связывают текст с контентом изображения.
4. **Оценки LLM (4–6 из 10)** означают, что модели создают осмысленные, но не всегда идеальные тексты.